In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import re
import numpy as np

Using your scraped data, investigates different relationships between candidates and the amount of money they raised. Here are some suggestions to get you started, but feel free to pose you own questions or do additional exploration:
a. How often does the candidate who raised more money win a race?
b. How often does the candidate who spent more money win a race?
c. Does the difference between either money raised or money spent seem to influence the likelihood of a candidate winning a race?
d. How often does the incumbent candidate win a race?
e. Can you detect any relationship between amount of money raised and the incumbent status of a candidate?

In [149]:
pd.options.display.max_rows=1000
All_states = pd.read_csv('All_states')
All_states[['Raised', 'Spent', 'Cash on Hand']] = All_states[['Raised', 'Spent', 'Cash on Hand']].replace('[\$,]', '', regex=True).astype(int)
All_states.drop(columns =['Last Report'], inplace=True)
All_states

,Candidate,Raised,Spent,Cash on Hand,District,State
0,Jerry Carl (R) • Winner,1971321,1859349,111973,1,AL
1,James Averhart (D),80095,78973,1122,1,AL
2,Barry Moore (R) • Winner,650807,669368,-13633,2,AL
3,Phyllis Harvey-Hall (D),56050,55988,0,2,AL
4,Mike D Rogers (R) • Incumbent • Winner,1193111,1218564,502234,3,AL
5,Adia Winfrey (D),50273,40971,43,3,AL
6,Robert B Aderholt (R) • Incumbent • Winner,1255076,1323812,647004,4,AL
7,Rick Neighbors (D),46068,45034,1034,4,AL
8,Mo Brooks (R) • Incumbent • Winner,655365,210045,1137501,5,AL
9,Gary Palmer (R) • Incumbent • Winner,907219,909082,370688,6,AL


In [192]:
#messing w groupby
state_district_subgroup = All_states.groupby(['District', 'State'])
for key, value in state_district_subgroup:
        print("---------")
        group = state_district_subgroup.get_group(key)
        print(group)
        

---------
                              Candidate   Raised    Spent  Cash on Hand  \
11  Don Young (R)  • Incumbent • Winner  1922901  1790448        249173   
12                     Alyse Galvin (I)  5178452  5088103         96594   

    District State  
11         1    AK  
12         1    AK  
---------
                  Candidate   Raised    Spent  Cash on Hand  District State
0  Jerry Carl (R)  • Winner  1971321  1859349        111973         1    AL
1        James Averhart (D)    80095    78973          1122         1    AL
---------
                                  Candidate  Raised    Spent  Cash on Hand  \
32  Rick Crawford (R)  • Incumbent • Winner  966801  1095518        182708   

    District State  
32         1    AR  
---------
                                   Candidate   Raised    Spent  Cash on Hand  \
13  Tom O'Halleran (D)  • Incumbent • Winner  3381353  3324138         63147   
14                         Tiffany Shedd (R)  1675618  1615704         63347   

   

520         1    NY  
---------
                                  Candidate   Raised    Spent  Cash on Hand  \
604  Steve Chabot (R)  • Incumbent • Winner  3177647  2861464         34508   
605                       Kate Schroder (D)  4040303  4039798           505   
606                          Kevin Kahn (L)    12085     8398          3687   

     District State  
604         1    OH  
605         1    OH  
606         1    OH  
---------
                                Candidate   Raised   Spent  Cash on Hand  \
639  Kevin Hern (R)  • Incumbent • Winner  1094104  830366        345198   
640                Kojo Asamoa-Caesar (D)   157355  173928         -5637   

     District State  
639         1    OK  
640         1    OK  
---------
                                      Candidate  Raised   Spent  Cash on Hand  \
648  Suzanne Bonamici (D)  • Incumbent • Winner  959223  864627        553346   
649                       Chris Christensen (R)     144     672             0   

    

373         2    ME  
---------
                                   Candidate   Raised    Spent  Cash on Hand  \
406  Bill Huizenga (R)  • Incumbent • Winner  2175411  1883610        469697   
407                       Bryan Berghoef (D)   751576   669864         19351   
408                 Jean-Michel Creviere (3)      281     1207          -559   

     District State  
406         2    MI  
407         2    MI  
408         2    MI  
---------
                                 Candidate   Raised    Spent  Cash on Hand  \
434  Angie Craig (D)  • Incumbent • Winner  5380101  4435492       1032000   
435                      Tyler Kistner (R)  2990054  2956042         34012   

     District State  
434         2    MN  
435         2    MN  
---------
                                Candidate   Raised    Spent  Cash on Hand  \
457  Ann Wagner (R)  • Incumbent • Winner  5598287  6513101        113443   
458                       Jill Schupp (D)  4850948  4830434         20513   

     D

44         3    CA  
---------
                        Candidate   Raised    Spent  Cash on Hand  District  \
142         Diane Mitsch Bush (D)  5008274  4984238         28455         3   
143  Lauren Boebert (R)  • Winner  2989510  2632716        356794         3   
144              Chris Milton (3)    12012    12012             0         3   

    State  
142    CO  
143    CO  
144    CO  
---------
                                    Candidate   Raised    Spent  Cash on Hand  \
160  Rosa L DeLauro (D)  • Incumbent • Winner  1938999  1795635        159728   
161                    Margaret Streicker (R)  1839080  1835385          3695   
162                        Justin Paglino (3)    29200    28670           529   

     District State  
160         3    CT  
161         3    CT  
162         3    CT  
---------
                     Candidate   Raised    Spent  Cash on Hand  District State
176  Kat Cammack (R)  • Winner  1122188  1090334         31853         3    FL
177       Ada

238         4    GA  
---------
                        Candidate   Raised    Spent  Cash on Hand  District  \
333             J.D. Scholten (D)  2885915  2902003         53112         4   
334  Randy Feenstra (R)  • Winner  1906234  1894288         11946         4   

    State  
333    IA  
334    IA  
---------
                                  Candidate  Raised   Spent  Cash on Hand  \
280  Jesus Garcia (D)  • Incumbent • Winner  822846  711159        126757   
281                    Jesus Solorio Jr (R)    2065     843          1129   

     District State  
280         4    IL  
281         4    IL  
---------
                               Candidate  Raised   Spent  Cash on Hand  \
314  Jim Baird (R)  • Incumbent • Winner  327727  190158        247371   
315                       Joe Mackey (D)   44585   43213          1590   

     District State  
314         4    IN  
315         4    IN  
---------
                               Candidate   Raised    Spent  Cash on Hand  \
3

...

194        10    FL  
---------
                               Candidate  Raised   Spent  Cash on Hand  \
256  Jody Hice (R)  • Incumbent • Winner  595947  457648        249916   
257            Tabitha Johnson-Green (D)   14808   12214            -8   

     District State  
256        10    GA  
257        10    GA  
---------
                                    Candidate   Raised    Spent  Cash on Hand  \
291  Brad Schneider (D)  • Incumbent • Winner  3459512  2873333       1873763   
292             Valerie Ramirez Mukherjee (R)   130225    64764         65461   

     District State  
291        10    IL  
292        10    IL  
---------
                      Candidate   Raised    Spent  Cash on Hand  District  \
422  Lisa McClain (R)  • Winner  2266287  2137031        123256        10   
423          Kimberly Bizon (D)    41438    41246         24582        10   

    State  
422    MI  
423    MI  
---------
                                     Candidate   Raised    Spent  \
594

261        12    GA  
---------
                               Candidate   Raised    Spent  Cash on Hand  \
295  Mike Bost (R)  • Incumbent • Winner  1918041  1509573        413789   
296                        Ray Lenzi (D)   161324   104809         57903   

     District State  
295        12    IL  
296        12    IL  
---------
                                    Candidate   Raised    Spent  Cash on Hand  \
426  Debbie Dingell (D)  • Incumbent • Winner  1630498  1689582        437383   
427                            Jeff Jones (R)    10616     9657          1513   

     District State  
426        12    MI  
427        12    MI  
---------
                                Candidate  Raised   Spent  Cash on Hand  \
598  Alma Adams (D)  • Incumbent • Winner  824124  651883        346673   
599                     Bill Brewster (R)   21950   16763          5187   

     District State  
598        12    NC  
599        12    NC  
---------
                                         

67        16    CA  
---------
                                     Candidate   Raised    Spent  \
205  Vernon Buchanan (R)  • Incumbent • Winner  3094611  3100766   
206                          Margaret Good (D)  3411924  3407845   

     Cash on Hand  District State  
205         73085        16    FL  
206          4080        16    FL  
---------
                                    Candidate   Raised    Spent  Cash on Hand  \
303  Adam Kinzinger (R)  • Incumbent • Winner  2105736  1588550       1523650   
304                       Dani Brzozowski (D)   459344   376167          -314   

     District State  
303        16    IL  
304        16    IL  
---------
                       Candidate   Raised    Spent  Cash on Hand  District  \
550  Jamaal Bowman (D)  • Winner  2878898  2801424         77474        16   

    State  
550    NY  
---------
                                      Candidate   Raised    Spent  \
637  Anthony Gonzalez (R)  • Incumbent • Winner  2338195  1839807 

77    CA  
---------
                                    Candidate   Raised    Spent  Cash on Hand  \
215  Lois J Frankel (D)  • Incumbent • Winner  1534882  1309786       1192171   
216                          Laura Loomer (R)  2271604  2240689         30915   
217                   Charleston Malkemus (I)    28928    28636           292   

     District State  
215        21    FL  
216        21    FL  
217        21    FL  
---------
                                    Candidate    Raised     Spent  \
561  Elise Stefanik (R)  • Incumbent • Winner  13287248  11655936   
562                            Tedra Cobb (D)   5586793   5593266   

     Cash on Hand  District State  
561       1983541        21    NY  
562         32848        21    NY  
---------
                              Candidate    Raised     Spent  Cash on Hand  \
776  Chip Roy (R)  • Incumbent • Winner   5005639   4679362        463583   
777                     Wendy Davis (D)  10263486  10201874         61611   

---------
                                 Candidate   Raised    Spent  Cash on Hand  \
800  John Carter (R)  • Incumbent • Winner  2160187  2214487        200406   
801                         Donna Imam (D)  1035222  1035272           -50   

     District State  
800        31    TX  
801        31    TX  
---------
                                     Candidate  Raised   Spent  Cash on Hand  \
97  Grace Napolitano (D)  • Incumbent • Winner  478426  251047        533330   
98                            Joshua Scott (R)   23624   14248         10155   

    District State  
97        32    CA  
98        32    CA  
---------
                                  Candidate   Raised    Spent  Cash on Hand  \
802  Colin Allred (D)  • Incumbent • Winner  5667725  5611747        159422   
803                   Genevieve Collins (R)  5923081  5915256          7825   
804                        Jason Sigmon (I)    11791    11651           154   

     District State  
802        32    TX  
803 

In [189]:
state, frame = next(iter(state_district_subgroup))
frame

,Candidate,Raised,Spent,Cash on Hand,District,State
11,Don Young (R) • Incumbent • Winner,1922901,1790448,249173,1,AK
12,Alyse Galvin (I),5178452,5088103,96594,1,AK


2

for each state and district(groupby?), if winner raised more money, True. Winner raised less money, False

In [148]:
All_states.groupby('State').nunique().iloc[:,-1].to_frame()

,District
State,
AK,1
AL,7
AR,4
AZ,9
CA,53
CO,7
CT,5
DE,1
FL,27


In [ ]:
All_states['Candidate'] = All_states['Candidate'].astype(str)
All_states['Candidate'].iat[0]


In [ ]:
386/431  #89.56% incumbents are winners
(All_states[All_states['Candidate'].str.contains("Incumbent")].count())/(All_states[All_states['Candidate'].str.contains("Winner")].count())